In [ ]:
import os
import csv
import requests
import pandas


# you won't be able to run this function without the api key in secret.txt on your desktop

def get_zip_from_the_goog(latitude,longitude):
    
    api_key = open('/Desktop/secret.txt','r').read()
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={apikey}'\
        .format(lat=latitude,lon=longitude,apikey=api_key)
    pull = requests.get(url)
    location_data = pull.json()['results'][0]['address_components']
    zip_code = None
    for entry in location_data:
        if entry['types'] == ['postal_code']: # verification becase the index of the zip changes sometimes!
            zip_code = entry['long_name']
    return zip_code


csvfile = open('StationEntrances.csv', newline='\n')
cr = list(csv.reader(csvfile, delimiter=',', quotechar='|'))[1:] # omit heading row
    
# store rows as tuples to be immutable in order to later typecast lat_lon_data_raw as a set to remove duplicates
lat_lon_data_raw = []
for row in cr:
    lat_lon_data_raw.append(tuple(row[:15]))

# converts tuples back to lists
lat_lon_data = []
for tup in list(set(lat_lon_data_raw)):
    lat_lon_data.append(list(tup))

# pass lat and lon to google api, append returned zip code to end of each row in lat_lon_data
for n, station in enumerate(lat_lon_data):
    zip_code = get_zip_from_the_goog(station[3],station[4])
    lat_lon_data[n].append(zip_code)


df = pandas.DataFrame(lat_lon_data)
df.to_csv('station_zips.csv')




